In [2]:
from redbox.graph.root import get_redbox_graph, run_redbox
from redbox.graph.chat import get_chat_with_docs_graph
from redbox.chains import components
from redbox.models.settings import Settings, ElasticLocalSettings, AISettings
from redbox.models.chain import ChainInput, ChainState
from pathlib import Path
from dotenv import load_dotenv, dotenv_values
from langchain_openai import AzureChatOpenAI
import tiktoken

_ = load_dotenv(Path.cwd().parent / ".env")

ENV = Settings(
    minio_host="localhost", 
    object_store="minio", 
    azure_openai_model="gpt-4o",
    elastic=ElasticLocalSettings(host="localhost"),
)

llm = AzureChatOpenAI(
    api_key=ENV.azure_openai_api_key,
    azure_endpoint=ENV.azure_openai_endpoint,
    model=ENV.azure_openai_model,
)

AI_CONFIG = {"configurable": ENV.ai.dict()}

## State

In [3]:
from uuid import UUID
from langchain_core.documents import Document
from redbox.models.chat import ChatRoute
from typing import Annotated, TypedDict

type DocumentState = dict[UUID, dict[UUID, Document]]

def document_reducer(left: DocumentState, right: DocumentState) -> DocumentState:
    """Merges two document states based on the following rules.
    
    * Groups are matched by the group key.
    * Documents are matched by the group key and document key.

    Then:
    
    * If key(s) are matched, the group or Document is replaced
    * If key(s) are matched and the key is None, the key is cleared
    * If key(s) aren't matched, group or Document is added
    """
    # If state is empty, return right
    if left is None:
        return right
    
    # Copy left
    reduced = {k: v.copy() for k, v in left.items()}
    
    # Update with right
    for group_key, group in right.items():
        # If group is None, remove from output if a group key is matched
        if group is None:
            reduced.pop(group_key, None)
            continue
        
        # If group key isn't matched, add it
        if group_key not in reduced:
            reduced[group_key] = group

        
        for document_key, document in group.items():
            if document is None:
                # If Document is None, remove from output if a group and document key is matched
                reduced[group_key].pop(document_key, None)
            else:
                # Otherwise, update or add the value
                reduced[group_key][document_key] = document

        # Remove group_key from output if it becomes empty after updates
        if not reduced[group_key]:
            del reduced[group_key]

    return reduced

class State(TypedDict):
    request: ChainInput
    text: str | None
    documents: Annotated[DocumentState, document_reducer] | None
    route: ChatRoute | None

In [4]:
x = {
    UUID("00000000-0000-0000-0000-000000000001"): {
        UUID("10000000-0000-0000-0000-000000000000"): Document(page_content="a"),
        UUID("20000000-0000-0000-0000-000000000000"): Document(page_content="b")
    },
    UUID("00000000-0000-0000-0000-000000000002"): {
        UUID("30000000-0000-0000-0000-000000000000"): Document(page_content="c"),
        UUID("40000000-0000-0000-0000-000000000000"): Document(page_content="d")
    }
}

y = {
    UUID("00000000-0000-0000-0000-000000000001"): {
        UUID("10000000-0000-0000-0000-000000000000"): Document(page_content="z"),
        UUID("20000000-0000-0000-0000-000000000000"): None,
        UUID("30000000-0000-0000-0000-000000000000"): Document(page_content="c")
    },
    UUID("00000000-0000-0000-0000-000000000002"): None
}

z = {
    UUID("00000000-0000-0000-0000-000000000001"): {
        UUID("10000000-0000-0000-0000-000000000000"): Document(page_content="z"),
        UUID("30000000-0000-0000-0000-000000000000"): Document(page_content="c")
    }
}

a = document_reducer(x, y)

a == z

True

## Edges

In [5]:
import tiktoken
import re

def calculate_token_budget(state: State, config: AISettings) -> int:
    tokeniser = tiktoken.get_encoding("cl100k_base")

    len_question = len(tokeniser.encode(state["request"].question))
    max_system_or_question_prompt = max(
        [
            len(tokeniser.encode(v))
            for k, v in config["configurable"].items() 
            if isinstance(v, str)
        ]
    )

    len_prompts = max_system_or_question_prompt * 2
    # config.llm_max_tokens is 1024 -- move max tokens to AISettings
    len_max_output = 1_024
    
    return config["configurable"]["context_window_size"] - len_max_output - len_prompts - len_question

def conditional_doceuments_bigger_than_context(state: State, config: AISettings) -> bool:
    token_budget = calculate_token_budget(state=state, config=config)

    if sum(d.metadata["token_count"] for d in state["documents"]) > token_budget:
        return True
    else:
        return False

def conditional_keyword_detection(state: State, config: AISettings) -> ChatRoute | None:
    re_keyword_pattern = re.compile(r"@(\w+)")

    route_match = re_keyword_pattern.search(state["request"].question)
    route_name = route_match.group()[1:] if route_match else None
    
    try:
        return ChatRoute[route_name]
    except KeyError:
        return None

def conditional_documents_selected(state: State, config: AISettings) -> bool:
    return len(state["request"].file_uuids) > 0

def conditional_multiple_docs_in_group(state: State, config: AISettings) -> bool:
    for group in state["documents"]:
        if len(state["documents"][group]) > 0:
            return True
    return False

## Sends

In [6]:
from langgraph.constants import Send
from typing import Callable

def make_document_group_send(target: str) -> Callable[[State, AISettings], list[Send]]:
    def _group_send(state: State, config: AISettings) -> list[Send]:
        group_send_states: list[State] = [
            State(
                request=state["request"],
                text=state["text"],
                documents={group_key: state["documents"][group_key]},
                route=state["route"]
            )
            for group_key in state["documents"]
        ]
        return [Send(node=target, arg=state) for state in group_send_states]
    return _group_send

def make_document_chunk_send(target: str) -> Callable[[State, AISettings], list[Send]]:
    def _chunk_send(state: State, config: AISettings) -> list[Send]:
        chunk_send_states: list[State] = [
            State(
                request=state["request"],
                text=state["text"],
                documents={group_key: {document_key: state["documents"][group_key][document_key]}},
                route=state["route"]
            )
            for group_key in state["documents"]
            for document_key in state["documents"][group_key]
        ]
        return [Send(node=target, arg=state) for state in chunk_send_states]
    return _chunk_send


## Patterns

In [7]:
def flatten_document_state(documents: dict[UUID, dict[UUID, Document]]) -> list[Document]:
    return [
        document
        for group in documents.values()
        for document in group.values()
    ]

In [8]:
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.retrievers import BaseRetriever

from redbox.api.runnables import make_chat_prompt_from_messages_runnable

from typing import Any

def make_chat_pattern(
    system_prompt: str,
    question_prompt: str,
    llm: BaseChatModel,
) -> Callable[[State, AISettings], dict[str, Any]]:
    """Returns a function that uses state["request"] to set state["text"]."""
    def _chat(state: State, config: AISettings) -> dict[str, Any]:
        chat_chain = (
            make_chat_prompt_from_messages_runnable(
                system_prompt=system_prompt,
                question_prompt=question_prompt,
                input_token_budget=config["configurable"]["context_window_size"] - 1_024,
                tokeniser=tiktoken.get_encoding("cl100k_base"),
            )
            | llm
            | StrOutputParser()
        )
        
        return {
            "text": chat_chain.invoke(state["request"]),
        }

    return _chat

chat_state = State(
    request=ChainInput(
        question="Who are you?",
        file_uuids=[],
        user_uuid="a16b2762-fc18-4595-bafa-3d70f1e85087",
        chat_history=[],
    ).dict(),
    text=None,
    documents=None,
    route=None
)

chat_node = make_chat_pattern(
    system_prompt="You're a pirate.", 
    question_prompt="{question}\n\n Response: ",
    llm=llm
)

chat_node(chat_state, AI_CONFIG)

{'text': "Arrr matey! I be Cap'n Quill, master of the seven seas and keeper of hidden treasures. What be yer business aboard me vessel?"}

In [10]:
def make_passthrough_pattern() -> Callable[[State, AISettings], dict[str, Any]]:
    """Returns a function that uses state["request"] to set state["text"]."""
    def _passthrough(state: State, config: AISettings) -> dict[str, Any]:
        return {
            "text": state["request"]["question"],
        }
    
    return _passthrough
    
passthrough_state = State(
    request=ChainInput(
        question="Who are you?",
        file_uuids=[],
        user_uuid="a16b2762-fc18-4595-bafa-3d70f1e85087",
        chat_history=[],
    ).dict(),
    text=None,
    documents=None,
    route=None
)

passthrough_node = make_passthrough_pattern()

passthrough_node(passthrough_state, AI_CONFIG)

{'text': 'Who are you?'}

In [11]:
def make_canned_pattern(canned: str) -> Callable[[State, AISettings], dict[str, Any]]:
    """Returns a function that uses state["request"] to set state["text"]."""
    def _canned(state: State, config: AISettings) -> dict[str, Any]:
        return {
            "text": canned,
        }
    
    return _canned
    
canned_state = State(
    request=ChainInput(
        question="Who are you?",
        file_uuids=[],
        user_uuid="a16b2762-fc18-4595-bafa-3d70f1e85087",
        chat_history=[],
    ).dict(),
    text=None,
    documents=None,
    route=None
)

canned_node = make_canned_pattern(canned="I am the walrus")

canned_node(canned_state, AI_CONFIG)

{'text': 'I am the walrus'}

In [12]:
from langchain_core.runnables import RunnablePassthrough 

from redbox.api.format import format_documents

from operator import itemgetter

def make_stuff_pattern(
    system_prompt: str,
    question_prompt: str,
    llm: BaseChatModel,
) -> Callable[[State, AISettings], dict[str, Any]]:
    """Returns a function that uses state["request"] and state["documents"] to set state["text"]."""
    def _stuff(state: State, config: AISettings) -> dict[str, Any]:
        stuff_chain = (
            RunnablePassthrough.assign(
                documents=(
                    RunnablePassthrough() 
                    | itemgetter("documents") 
                    | flatten_document_state 
                    | format_documents
                )
            ) 
            | make_chat_prompt_from_messages_runnable(
                system_prompt=system_prompt,
                question_prompt=question_prompt,
                input_token_budget=config["configurable"]["context_window_size"] - 1_024,
                tokeniser=tiktoken.get_encoding("cl100k_base"),
            )
            | llm
            | StrOutputParser()
        )
        
        return {
            "text": stuff_chain.invoke({
                **state["request"],
                "documents": state["documents"]
            })
        }

    return _stuff

stuff_state = State(
    request=ChainInput(
        question="What animals are named in these documents?",
        file_uuids=["71b76e2a-9c2f-4340-ba94-cc09c7de74ff"],
        user_uuid="a16b2762-fc18-4595-bafa-3d70f1e85087",
        chat_history=[],
    ).dict(),
    text=None,
    documents={
        UUID("00000000-0000-0000-0000-000000000001"): {
            UUID("10000000-0000-0000-0000-000000000000"): Document(
                page_content="A dog is a type of mammal.",
                metadata={
                    "uuid": UUID("10000000-0000-0000-0000-000000000000"),
                    "parent_file_uuid": UUID("00000000-0000-0000-0000-000000000001"),
                    "token_count": 6,
                }
            ),
            UUID("20000000-0000-0000-0000-000000000000"): Document(
                page_content="A cat is a type of mammal.",
                metadata={
                    "uuid": UUID("20000000-0000-0000-0000-000000000000"),
                    "parent_file_uuid": UUID("00000000-0000-0000-0000-000000000001"),
                    "token_count": 6,
                }
            )
        },
        UUID("00000000-0000-0000-0000-000000000002"): {
            UUID("30000000-0000-0000-0000-000000000000"): Document(
                page_content="A swift is a type of bird.",
                metadata={
                    "uuid": UUID("30000000-0000-0000-0000-000000000000"),
                    "parent_file_uuid": UUID("00000000-0000-0000-0000-000000000002"),
                    "token_count": 6,
                }
            ),
            UUID("40000000-0000-0000-0000-000000000000"): Document(
                page_content="A shrike is a type of bird.",
                metadata={
                    "uuid": UUID("40000000-0000-0000-0000-000000000000"),
                    "parent_file_uuid": UUID("00000000-0000-0000-0000-000000000002"),
                    "token_count": 6,
                }
            )
        }
    },
    route=None
)

stuff_node = make_stuff_pattern(
    system_prompt="Summarise these documents.", 
    question_prompt="Question: {question}. \n\n Documents: \n\n {documents} \n\n Answer:",
    llm=llm
)

stuff_node(stuff_state, AI_CONFIG)

{'text': 'The animals named in the documents are:\n\n1. Dog\n2. Cat\n3. Swift\n4. Shrike'}

In [13]:
from redbox.transform import combine_documents

from functools import reduce
from uuid import uuid4

def make_merge_pattern(
    system_prompt: str,
    question_prompt: str,
    llm: BaseChatModel,
) -> Callable[[State, AISettings], dict[str, Any]]:
    """Returns a function that uses state["request"] and state["documents"] to return one item in state["documents"].
    
    When combined with chunk send, will replace each Document with what's returned from the LLM.

    When combined with group end, with combine all Documents and use the metadata of the first.

    When used without a send, the first Document receieved defines the metadata.
    """
    def _merge(state: State, config: AISettings) -> dict[str, Any]:
        tokeniser = tiktoken.get_encoding("cl100k_base")

        flattened_documents = flatten_document_state(state["documents"])
        
        merged_document = reduce(lambda l, r: combine_documents(l, r), flattened_documents)
        
        stuff_chain = (
            RunnablePassthrough.assign(
                documents=(
                    RunnablePassthrough() 
                    | itemgetter("documents") 
                    | flatten_document_state 
                    | format_documents
                )
            ) 
            | make_chat_prompt_from_messages_runnable(
                system_prompt=system_prompt,
                question_prompt=question_prompt,
                input_token_budget=config["configurable"]["context_window_size"] - 1_024,
                tokeniser=tokeniser,
            )
            | llm
            | StrOutputParser()
        )
        
        merged_document.page_content = stuff_chain.invoke({
            **state["request"],
            "documents": state["documents"]
        })
        merged_document.metadata["token_count"] = len(
            tokeniser.encode(merged_document.page_content)
        )

        group_uuid = merged_document.metadata.get("parent_file_uuid", uuid4())
        document_uuid = merged_document.metadata.get("uuid", uuid4())

        # Clear old documents, add new one
        document_state = state["documents"].copy()
        
        for group in document_state:
            for document in document_state[group]:
                document_state[group][document] = None
        
        document_state[group_uuid][document_uuid] = merged_document
        
        return {
            "documents": document_state
        }

    return _merge

merge_state = State(
    request=ChainInput(
        question="What animals are named in these documents?",
        file_uuids=["71b76e2a-9c2f-4340-ba94-cc09c7de74ff"],
        user_uuid="a16b2762-fc18-4595-bafa-3d70f1e85087",
        chat_history=[],
    ).dict(),
    text=None,
    documents={
        UUID("00000000-0000-0000-0000-000000000001"): {
            UUID("10000000-0000-0000-0000-000000000000"): Document(
                page_content="A dog is a type of mammal.",
                metadata={
                    "uuid": UUID("10000000-0000-0000-0000-000000000000"),
                    "parent_file_uuid": UUID("00000000-0000-0000-0000-000000000001"),
                    "token_count": 6,
                }
            ),
            UUID("20000000-0000-0000-0000-000000000000"): Document(
                page_content="A cat is a type of mammal.",
                metadata={
                    "uuid": UUID("20000000-0000-0000-0000-000000000000"),
                    "parent_file_uuid": UUID("00000000-0000-0000-0000-000000000001"),
                    "token_count": 6,
                }
            )
        },
        UUID("00000000-0000-0000-0000-000000000002"): {
            UUID("30000000-0000-0000-0000-000000000000"): Document(
                page_content="A swift is a type of bird.",
                metadata={
                    "uuid": UUID("30000000-0000-0000-0000-000000000000"),
                    "parent_file_uuid": UUID("00000000-0000-0000-0000-000000000002"),
                    "token_count": 6,
                }
            ),
            UUID("40000000-0000-0000-0000-000000000000"): Document(
                page_content="A shrike is a type of bird.",
                metadata={
                    "uuid": UUID("40000000-0000-0000-0000-000000000000"),
                    "parent_file_uuid": UUID("00000000-0000-0000-0000-000000000002"),
                    "token_count": 6,
                }
            )
        }
    },
    route=None
)

merge_node = make_merge_pattern(
    system_prompt="Summarise these documents.", 
    question_prompt="Question: {question}. \n\n Documents: \n\n {documents} \n\n Answer:",
    llm=llm
)

merge_node(merge_state, AI_CONFIG)


{'documents': {UUID('00000000-0000-0000-0000-000000000001'): {UUID('10000000-0000-0000-0000-000000000000'): Document(metadata={'uuid': UUID('10000000-0000-0000-0000-000000000000'), 'parent_file_uuid': UUID('00000000-0000-0000-0000-000000000001'), 'token_count': 24, 'page_number': None, 'languages': None, 'link_texts': None, 'link_urls': None, 'links': None}, page_content='The animals named in the documents are:\n\n1. Dog\n2. Cat\n3. Swift\n4. Shrike'),
   UUID('20000000-0000-0000-0000-000000000000'): None},
  UUID('00000000-0000-0000-0000-000000000002'): {UUID('30000000-0000-0000-0000-000000000000'): None,
   UUID('40000000-0000-0000-0000-000000000000'): None}}}

In [14]:
def make_retrieve_pattern(
    retriever: BaseRetriever
) -> Callable[[State, AISettings], dict[str, Any]]:
    """Returns a function that uses state["request"] and state["text"] to set state["documents"]."""
    def _retrieve(state: State, config: AISettings) -> dict[str, Any]:
        document_states: list[DocumentState] = []
        request = state["request"]
        request["question"] = state["text"]

        for document in retriever.invoke(request):
            group_uuid = document.metadata.get("parent_file_uuid", uuid4())
            chunk_uuid = document.metadata.get("uuid", uuid4())
            document_states.append({ group_uuid: { chunk_uuid: document } })
    
        return {
            "documents": reduce(lambda l, r: document_reducer(l, r), document_states)
        }

    return _retrieve


class FakeRetriever(BaseRetriever):
    docs: list[Document]

    def _get_relevant_documents(self, query: str) -> list[Document]:
        return self.docs

    async def _aget_relevant_documents(self, query: str) -> list[Document]:
        return self.docs


fake_retriever = FakeRetriever(
    docs=[
        Document(
            page_content="A dog is a type of mammal.",
            metadata={
                "uuid": UUID("10000000-0000-0000-0000-000000000000"),
                "parent_file_uuid": UUID("00000000-0000-0000-0000-000000000001"),
                "token_count": 6,
            }
        ),
        Document(
            page_content="A cat is a type of mammal.",
            metadata={
                "uuid": UUID("20000000-0000-0000-0000-000000000000"),
                "parent_file_uuid": UUID("00000000-0000-0000-0000-000000000001"),
                "token_count": 6,
            }
        ),
        Document(
            page_content="A swift is a type of bird.",
            metadata={
                "uuid": UUID("30000000-0000-0000-0000-000000000000"),
                "parent_file_uuid": UUID("00000000-0000-0000-0000-000000000002"),
                "token_count": 6,
            }
        ),
        Document(
            page_content="A shrike is a type of bird.",
            metadata={
                "uuid": UUID("40000000-0000-0000-0000-000000000000"),
                "parent_file_uuid": UUID("00000000-0000-0000-0000-000000000002"),
                "token_count": 6,
            }
        )
    ]
)

retrieve_state = State(
    request=ChainInput(
        question="What are some animals?",
        file_uuids=["71b76e2a-9c2f-4340-ba94-cc09c7de74ff"],
        user_uuid="a16b2762-fc18-4595-bafa-3d70f1e85087",
        chat_history=[],
    ).dict(),
    text=None,
    documents=None,
    route=None
)

retrieve_node = make_retrieve_pattern(retriever=fake_retriever)

document_state = retrieve_node(retrieve_state, AI_CONFIG)

for group in document_state["documents"]:
    print(f"Group:{group}")
    for document in document_state["documents"][group]:
        print(f"Document:{document}")
    print("---")

Group:00000000-0000-0000-0000-000000000001
Document:10000000-0000-0000-0000-000000000000
Document:20000000-0000-0000-0000-000000000000
---
Group:00000000-0000-0000-0000-000000000002
Document:30000000-0000-0000-0000-000000000000
Document:40000000-0000-0000-0000-000000000000
---


## Subgraphs

In [15]:
from langgraph.graph import StateGraph, START, END

chat = StateGraph(State)

chat.add_node(
    "chat",
    make_chat_pattern(
        system_prompt="You're a pirate.", 
        question_prompt="{question}\n\n Response: ",
        llm=llm
    )
)

chat.add_edge(START, "chat")
chat.add_edge("chat", END)

chat_compiled = chat.compile()

chat_state = State(
    request=ChainInput(
        question="What are some animals?",
        file_uuids=["71b76e2a-9c2f-4340-ba94-cc09c7de74ff"],
        user_uuid="a16b2762-fc18-4595-bafa-3d70f1e85087",
        chat_history=[
            {"role": "user", "text": "Who are ye?"},
            {"role": "ai", "text": "Avast ye I'm Blackbeard!"},
        ],
    ).dict(),
    text=None,
    documents=None,
    route=None
)

chat_compiled.invoke(input=chat_state, config=AI_CONFIG)


{'request': {'question': 'What are some animals?',
  'file_uuids': [UUID('71b76e2a-9c2f-4340-ba94-cc09c7de74ff')],
  'user_uuid': UUID('a16b2762-fc18-4595-bafa-3d70f1e85087'),
  'chat_history': [{'role': 'user', 'text': 'Who are ye?'},
   {'role': 'ai', 'text': "Avast ye I'm Blackbeard!"}]},
 'text': "Arrr, there be many creatures on land and sea! Ye've got the mighty whale, the cunning fox, the fierce tiger, and the swift eagle, just to name a few. What be ye interested in, matey?",
 'documents': None,
 'route': None}

In [16]:
from langgraph.graph import StateGraph, START, END

rag = StateGraph(State)

rag.add_node(
    "rag_chat",
    make_chat_pattern(
        system_prompt="Write the following chat as a single question.", 
        question_prompt="{question}\n\n Single question: ",
        llm=llm
    )
)
rag.add_node(
    "rag_retrieve", 
    make_retrieve_pattern(retriever=fake_retriever)
)
rag.add_node(
    "rag_stuff",
    make_stuff_pattern(
        system_prompt="Summarise these documents.", 
        question_prompt="Question: {question}. \n\n Documents: \n\n {documents} \n\n Answer:",
        llm=llm
    )
)

rag.add_edge(START, "rag_chat")
rag.add_edge("rag_chat", "rag_retrieve")
rag.add_edge("rag_retrieve", "rag_stuff")
rag.add_edge("rag_stuff", END)

rag_compiled = rag.compile()

rag_state = State(
    request=ChainInput(
        question="What are some animals?",
        file_uuids=["71b76e2a-9c2f-4340-ba94-cc09c7de74ff"],
        user_uuid="a16b2762-fc18-4595-bafa-3d70f1e85087",
        chat_history=[],
    ).dict(),
    text=None,
    documents=None,
    route=None
)

rag_compiled.invoke(input=rag_state, config=AI_CONFIG)


{'request': {'question': 'What are some examples of different types of animals?',
  'file_uuids': [UUID('71b76e2a-9c2f-4340-ba94-cc09c7de74ff')],
  'user_uuid': UUID('a16b2762-fc18-4595-bafa-3d70f1e85087'),
  'chat_history': []},
 'text': 'Examples of different types of animals include:\n\n- Dogs and cats, which are types of mammals.\n- Swifts and shrikes, which are types of birds.',
 'documents': {UUID('00000000-0000-0000-0000-000000000001'): {UUID('10000000-0000-0000-0000-000000000000'): Document(metadata={'uuid': UUID('10000000-0000-0000-0000-000000000000'), 'parent_file_uuid': UUID('00000000-0000-0000-0000-000000000001'), 'token_count': 6}, page_content='A dog is a type of mammal.'),
   UUID('20000000-0000-0000-0000-000000000000'): Document(metadata={'uuid': UUID('20000000-0000-0000-0000-000000000000'), 'parent_file_uuid': UUID('00000000-0000-0000-0000-000000000001'), 'token_count': 6}, page_content='A cat is a type of mammal.')},
  UUID('00000000-0000-0000-0000-000000000002'): {UU

In [58]:
from langgraph.graph import StateGraph, START, END

cwd = StateGraph(State)

cwd.add_node(
    "cwd_passthrough",
    make_passthrough_pattern()
)
cwd.add_node(
    "cwd_retrieve", 
    make_retrieve_pattern(retriever=fake_retriever)
)
cwd.add_node(
    "cwd_stuff",
    make_stuff_pattern(
        system_prompt="Summarise these documents.", 
        question_prompt="Question: {question}. \n\n Documents: \n\n {documents} \n\n Answer:",
        llm=llm
    )
)
cwd.add_node(
    "cwd_chunk_send_to_shrink",
    make_document_chunk_send("cwd_shrink")
)
cwd.add_node(
    "cwd_shrink",
    make_merge_pattern(
        system_prompt="Summarise these documents.", 
        question_prompt="Question: {question}. \n\n Documents: \n\n {documents} \n\n Answer:",
        llm=llm
    )
)
cwd.add_node(
    "cwd_canned_error",
    make_chat_canned_pattern(canned="These documents are too large to work with.")
)
cwd.add_node(
    "cwd_group_send_to_merge",
    make_document_group_send("cwd_group_merge")
)
cwd.add_node(
    "cwd_group_merge",
    make_merge_pattern(
        system_prompt="Summarise these documents.", 
        question_prompt="Question: {question}. \n\n Documents: \n\n {documents} \n\n Answer:",
        llm=llm
    )
)

cwd.add_edge(START, "cwd_passthrough")
cwd.add_edge("cwd_passthrough", "cwd_retrieve")
cwd.add_conditional_edges(
    "cwd_retrieve", 
    conditional_doceuments_bigger_than_context, 
    {True: "cwd_chunk_send_to_shrink", False: "cwd_stuff"}
)
cwd.add_conditional_edges(
    "cwd_shrink",
    conditional_multiple_docs_in_group,
    {True: "cwd_group_send_to_merge", False: "cwd_stuff"}
)
cwd.add_edge("cwd_group_merge", "cwd_stuff")
cwd.add_edge("cwd_stuff", END)

cwd_compiled = cwd.compile()

cwd_state = State(
    request=ChainInput(
        question="What are some animals?",
        file_uuids=["71b76e2a-9c2f-4340-ba94-cc09c7de74ff"],
        user_uuid="a16b2762-fc18-4595-bafa-3d70f1e85087",
        chat_history=[],
    ).dict(),
    text=None,
    documents=None,
    route=None
)

cwd_compiled.invoke(input=cwd_state, config=AI_CONFIG)


ValueError: Node `cwd_shrink` is not reachable

## Graph

In [59]:
# TODO